## Scrape the Location Counts

### 1. Use the beautifulsoup library to scrape the data (from the link above) on state names and corresponding number of store locations, for the following chains:
* Starbucks

* Dunkin’ Donuts

In [62]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup

In [63]:
# starbucks
response = requests.get("https://worldpopulationreview.com/state-rankings/starbucks-stores-by-state")
starbucks_soup = BeautifulSoup(response.content, "html.parser")
starbucks_tables = starbucks_soup.find_all("table", class_ = "wpr-table")[0]


In [130]:
# create empty lists for the dictionaries
starbucks_rows = []
states_list = []

for th in starbucks_tables.find_all("th")[4:]:
    states = th.get_text(strip = True)
    states_list.append(states)

for i, tr in enumerate(starbucks_tables.find_all("tr")[1:]):
    state = states_list[i]

    cells = tr.find_all("td")
    
    # extract cells for stores in 2023, 2021, and 2024
    stores2023 = cells[0].get_text(strip=True)
    stores2021 = cells[1].get_text(strip=True)
    stores2024 = cells[2].get_text(strip=True)

    # append the rows with the dictionaries
    starbucks_rows.append({
        "State": state,
        "Year": 2023,
        "Starbucks Stores": stores2023, 
        "Company": "Starbucks"
    })
    starbucks_rows.append({
        "State": state,
        "Year": 2021,
        "Starbucks Stores": stores2021,
        "Company": "Starbucks"
    })
    starbucks_rows.append({
        "State": state,
        "Year": 2024,
        "Starbucks Stores": stores2024, 
        "Company": "Starbucks"
    })

# convert into a data frame
df = pd.DataFrame(starbucks_rows)
print(df)

            State  Year Starbucks Stores    Company
0      California  2023            3,080  Starbucks
1      California  2021            2,959  Starbucks
2      California  2024            3,117  Starbucks
3           Texas  2023            1,346  Starbucks
4           Texas  2021            1,215  Starbucks
..            ...   ...              ...        ...
148  North Dakota  2021               20  Starbucks
149  North Dakota  2024                   Starbucks
150       Vermont  2023                8  Starbucks
151       Vermont  2021               35  Starbucks
152       Vermont  2024                   Starbucks

[153 rows x 4 columns]
            State  Year Starbucks Stores    Company
0      California  2023            3,080  Starbucks
1      California  2021            2,959  Starbucks
2      California  2024            3,117  Starbucks
3           Texas  2023            1,346  Starbucks
4           Texas  2021            1,215  Starbucks
..            ...   ...              ...

In [131]:
#dunkin
dunkin_response = requests.get("https://worldpopulationreview.com/state-rankings/dunkin-donuts-by-state")
dunkin_soup = BeautifulSoup(dunkin_response.content, "html.parser")
dunkin_table = dunkin_soup.find("table", class_ = "wpr-table")

In [132]:
# create empty lists for the dictionaries
dunkin_rows = []
states_list = []

for th in dunkin_table.find_all("th")[3:]:
    states = th.get_text(strip = True)
    states_list.append(states)

for i, tr in enumerate(dunkin_table.find_all("tr")[1:]):
    state = states_list[i]

    cells = tr.find_all("td")

    # extract cells for stores
    dunkin_stores2024 = cells[0].get_text(strip=True)
    dunkin_stores2023 = cells[1].get_text(strip=True)

    #append the rows with the dictionaries
    dunkin_rows.append({
        "State": state,
        "Year": 2024,
        "Dunkin Stores": dunkin_stores2024,
        "Company": "Dunkin"
    })
    dunkin_rows.append({
        "State": state,
        "Year": 2023,
        "Dunkin Stores": dunkin_stores2023,
        "Company": "Dunkin"
    })
# convert to data frame
df = pd.DataFrame(dunkin_rows)
print(df)

             State  Year Dunkin Stores Company
0         New York  2024         1,431  Dunkin
1         New York  2023         1,414  Dunkin
2    Massachusetts  2024         1,042  Dunkin
3    Massachusetts  2023         1,068  Dunkin
4          Florida  2024           909  Dunkin
..             ...   ...           ...     ...
97          Oregon  2023             0  Dunkin
98    South Dakota  2024             0  Dunkin
99    South Dakota  2023             0  Dunkin
100     Washington  2024             0  Dunkin
101     Washington  2023            19  Dunkin

[102 rows x 4 columns]
             State  Year Dunkin Stores Company
0         New York  2024         1,431  Dunkin
1         New York  2023         1,414  Dunkin
2    Massachusetts  2024         1,042  Dunkin
3    Massachusetts  2023         1,068  Dunkin
4          Florida  2024           909  Dunkin
..             ...   ...           ...     ...
97          Oregon  2023             0  Dunkin
98    South Dakota  2024            

### 2. Parse, merge and tidy your data. Think carefully about what the tidy version of this dataset is with multiple years represented on the website.

In [134]:
# rename the dataframes
df_dunkin = pd.DataFrame(dunkin_rows)
df_starbucks = pd.DataFrame(starbucks_rows)

#merge two data frames
coffee = pd.merge(df_dunkin, df_starbucks, on=["State", "Year", "Company"], how="outer")
print(coffee)


       State  Year Dunkin Stores    Company Starbucks Stores
0    Alabama  2021           NaN  Starbucks               99
1    Alabama  2023            59     Dunkin              NaN
2    Alabama  2023           NaN  Starbucks               85
3    Alabama  2024            69     Dunkin              NaN
4    Alabama  2024           NaN  Starbucks                 
..       ...   ...           ...        ...              ...
250  Wyoming  2021           NaN  Starbucks               26
251  Wyoming  2023             1     Dunkin              NaN
252  Wyoming  2023           NaN  Starbucks               23
253  Wyoming  2024             1     Dunkin              NaN
254  Wyoming  2024           NaN  Starbucks                 

[255 rows x 5 columns]
       State  Year Dunkin Stores    Company Starbucks Stores
0    Alabama  2021           NaN  Starbucks               99
1    Alabama  2023            59     Dunkin              NaN
2    Alabama  2023           NaN  Starbucks               85


## Supplemental Data

### 4. Scrape the state names and populations from this wikipedia page. Merge these data with your coffee dataset.

In [136]:
pop_response = requests.get("https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population")
pop_soup = BeautifulSoup(pop_response.content, "html.parser")
pop_table = pop_soup.find("table", class_ = "wikitable")

pop_rows = []

for tr in pop_table.find_all("tr")[1:]:
    cells = tr.find_all('td')
    if len(cells) >= 3:
        state = cells[2].get_text(strip = True)
        population = cells[3].get_text(strip = True)


        pop_rows.append({
        "State": state,
        "Population": population})

# convert to data frame
pop_df = pd.DataFrame(pop_rows)

# merge two data frames
coffee_new = pd.merge(coffee, pop_df, on="State", how = "outer")
print(coffee_new)


       State    Year Dunkin Stores    Company Starbucks Stores Population
0    Alabama  2021.0           NaN  Starbucks               99  5,024,279
1    Alabama  2023.0            59     Dunkin              NaN  5,024,279
2    Alabama  2023.0           NaN  Starbucks               85  5,024,279
3    Alabama  2024.0            69     Dunkin              NaN  5,024,279
4    Alabama  2024.0           NaN  Starbucks                   5,024,279
..       ...     ...           ...        ...              ...        ...
259  Wyoming  2021.0           NaN  Starbucks               26    576,851
260  Wyoming  2023.0             1     Dunkin              NaN    576,851
261  Wyoming  2023.0           NaN  Starbucks               23    576,851
262  Wyoming  2024.0             1     Dunkin              NaN    576,851
263  Wyoming  2024.0           NaN  Starbucks                     576,851

[264 rows x 6 columns]
       State    Year Dunkin Stores    Company Starbucks Stores Population
0    Alabama  

### 5. Find the revenue, stock price, or your financial metric of choice for each of the companies listed above (if you can find a website to scrape these from that’s great!…but it’s okay if you manually enter these). Merge these values into your big dataset. Note: these values may be repeated for each state.

In [137]:
# stock prices
# 97.31 for starbucks
# 106.48 for dunkin 
financial_data = {
    "Company": ["Starbucks", "Dunkin"],
    "Stock Price": [97.31, 106.48]
}

financial_df = pd.DataFrame(financial_data)

financial_coffee = pd.merge(coffee_new, financial_df, on = "Company", how = "outer")
financial_coffee = financial_coffee.sort_values(by = "State").reset_index(drop=True)
print(financial_coffee)

       State    Year Dunkin Stores  ... Starbucks Stores Population Stock Price
0    Alabama  2023.0            59  ...              NaN  5,024,279      106.48
1    Alabama  2024.0           NaN  ...                   5,024,279       97.31
2    Alabama  2023.0           NaN  ...               85  5,024,279       97.31
3    Alabama  2021.0           NaN  ...               99  5,024,279       97.31
4    Alabama  2024.0            69  ...              NaN  5,024,279      106.48
..       ...     ...           ...  ...              ...        ...         ...
259  Wyoming  2021.0           NaN  ...               26    576,851       97.31
260  Wyoming  2023.0           NaN  ...               23    576,851       97.31
261  Wyoming  2024.0           NaN  ...                     576,851       97.31
262  Wyoming  2023.0             1  ...              NaN    576,851      106.48
263  Wyoming  2024.0             1  ...              NaN    576,851      106.48

[264 rows x 7 columns]
       State    

### 6. Create a region variable in your dataset according to the scheme on this wikipedia page: Northeast, Midwest, South, West. You do not need to scrape this information.

In [138]:
#regions for us
regions = {
    "Conneticut": "Northeast",
    "Maine": "Northeast",
    "Massachusetts": "Northeast",
    "New Hampshire": "Northeast",
    "Rhode Island": "Northeast",
    "Vermont": "Northeast",
    "New Jersey": "Northeast",
    "New York": "Northeast",
    "Pennsylvania": "Northeast",
    "Illinois": "Midwest",
    "Indiana": "Midwest",
    "Michigan": "Midwest",
    "Ohio": "Midwest",
    "Wisconsin": "Midwest",
    "Iowa": "Midwest",
    "Kansas": "Midwest",
    "Minnesota": "Midwest",
    "Missouri": "Midwest",
    "Nebraska": "Midwest",
    "North Dakota": "Midwest",
    "South Dakota": "Midwest",
    "Delaware": "South",
    "Florida": "South",
    "Georgia": "South",
    "Maryland": "South",
    "North Carolina": "South",
    "South Carolina": "South",
    "Virginia": "South",
    "District of Columbia": "South",
    "West Virginia": "South",
    "Alabama": "South",
    "Kentucky": "South",
    "Mississippi": "South",
    "Tennessee": "South",
    "Arkansas": "South",
    "Louisiana": "South",
    "Oklahoma": "South",
    "Texas": "South",
    "Arizona": "West",
    "Colorado": "West",
    "Idaho": "West",
    "Montana": "West", 
    "Nevada": "West", 
    "New Mexico": "West",
    "Utah": "West",
    "Wyoming": "West",
    "Alaska": "West",
    "California": "West",
    "Hawaii": "West",
    "Oregon": "West",
    "Washington": "West"
}

financial_coffee["Regions"] = financial_coffee["State"].map(regions)

In [139]:
financial_coffee["Starbucks Stores"] = pd.to_numeric(financial_coffee["Starbucks Stores"], errors='coerce')
financial_coffee["Dunkin Stores"] = pd.to_numeric(financial_coffee["Dunkin Stores"], errors='coerce')

## Analyze

### 7. Assess and comment on the prevalence of each chain. Some questions to consider (you don’t need to answer all of these and you may come up with your own):

Based on the dataframe, we cannot fully compare the two chains becasue the Starbucks data includes information on years 2021, 2023, and 2024. Dunkin contains information on years 2023 and 2024. A conclusion could be drawn that population is somewhat correlated witht the number of Starbucks Stores. For example, Florida has the highest number of Starbucks stores at 892 stores for 2024 with a population of 21 million, while Vermont has the lowest amount of stores with 8 for 2023 with a population of 643k. The same conclusion could be drawn for Dunkin Stores with Florida having the highest number of stores at 909 with a population of 21 million, while Wyoming has 1 store with a population of 576k. Dunkin is more popular in the South and Northeast regions with there being a higher number of stores in those regions than the others. Starbucks is more popular in the West and South regions with there being a higher number of stores in those regions than others. The financial data does not change from state to state or region to region because it is the current stock price for each company and is universal across the country. 

## Automate

### Convert your code for Exercises 1-3 above to a function that takes a single argument: the URL. This function should

* Scrape the information on state names and corresponding number of store locations on the webpage specified (assume the page has a table in the same form and placement as the ones you scraped above)

* Extract the name of the company from either the URL specified or the webpage (assume the URL will have the same format as the ones used above)

* Return a clean, organized and tidy dataset. Find a page other than Starbucks and Dunkin’ Donuts to test this on to confirm that it works. It’s fine if this is not related to coffee.

In [140]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def scrape_store_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    company_name = re.search(r'state-rankings/(.*?)-by-state', url).group(1).replace('-', ' ').title()
    table = soup.find('table', class_ = 'wpr-table')
    states_list = [th.get_text(strip = True) for th in table.find_all('th')[3:]]
    rows = []
    for i, tr in enumerate(table.find_all('tr')[1:]):
        #state = states_list[i]
        cells = tr.find_all('td')
        if i < len(states_list):
            state = states_list[i]
        else: 
            continue

        stores_2023 = cells[0].get_text(strip = True)
        stores_2021 = cells[1].get_text(strip = True) if len(cells) > 1 else None
        stores_2024 = cells[2].get_text(strip = True) if len(cells) > 2 else None

        if stores_2023:
            rows.append({"State": state, "Year": 2023, "Stores": stores_2023, "Company": company_name})
        if stores_2021:
            rows.append({"State": state, "Year": 2021, "Stores": stores_2021, "Company": company_name})
        if stores_2024:
            rows.append({"State": state, "Year": 2024, "Stores": stores_2024, "Company": company_name})

    df = pd.DataFrame(rows)
    df["Stores"] = pd.to_numeric(df["Stores"], errors='coerce')
    
    return df


In [128]:
def scrape_store_data2(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    company_name = re.search(r'state-rankings/(.*?)-by-state', url).group(1).replace('-', ' ').title()
    table = soup.find('table', class_ = 'wpr-table')
    states_list = [th.get_text(strip = True) for th in table.find_all('th')[3:]]
    headers = []
    rows = []
    states_list = []
    for th in table.find_all('th', class_ = 'datatable-th'):
        header = th.get_text(strip = True)
        headers.append(header)
    index = len(table.find_all('th', class_ = "datatable-th")) 
    
    for th in table.find_all('th')[index:]:
        states = th.get_text(strip = True)
        states_list.append(states)

        stores_2023 = cells[0].get_text(strip = True)
        stores_2021 = cells[1].get_text(strip = True) if len(cells) > 1 else None
        stores_2024 = cells[2].get_text(strip = True) if len(cells) > 2 else None

        if stores_2023:
            rows.append({"State": state, "Year": 2023, "Stores": stores_2023, "Company": company_name})
        if stores_2021:
            rows.append({"State": state, "Year": 2021, "Stores": stores_2021, "Company": company_name})
        if stores_2024:
            rows.append({"State": state, "Year": 2024, "Stores": stores_2024, "Company": company_name})

    df = pd.DataFrame(rows)
    df["Stores"] = pd.to_numeric(df["Stores"], errors='coerce')
    
    return df

In [129]:
# test function
url = "https://worldpopulationreview.com/state-rankings/chick-fil-a-by-state"
scrape_store_data2(url)

,State,Year,Stores,Company
0,Total U.S. (including D.C. and territories),2023,NaN,Chick Fil A
1,Total U.S. (including D.C. and territories),2021,NaN,Chick Fil A
2,Total U.S. (including D.C. and territories),2024,NaN,Chick Fil A
3,Total U.S. (including D.C. and territories),2023,NaN,Chick Fil A
4,Total U.S. (including D.C. and territories),2021,NaN,Chick Fil A
...,...,...,...,...
148,Total U.S. (including D.C. and territories),2021,NaN,Chick Fil A
149,Total U.S. (including D.C. and territories),2024,NaN,Chick Fil A
150,Total U.S. (including D.C. and territories),2023,NaN,Chick Fil A
151,Total U.S. (including D.C. and territories),2021,NaN,Chick Fil A


In [120]:
# test function
url = "https://worldpopulationreview.com/state-rankings/starbucks-stores-by-state"
scrape_store_data(url)

,State,Year,Stores,Company
0,Starbucks Stores 2024,2023,NaN,Starbucks Stores
1,Starbucks Stores 2024,2021,NaN,Starbucks Stores
2,Starbucks Stores 2024,2024,NaN,Starbucks Stores
3,California,2023,NaN,Starbucks Stores
4,California,2021,NaN,Starbucks Stores
...,...,...,...,...
106,West Virginia,2021,26.0,Starbucks Stores
107,Wyoming,2023,13.0,Starbucks Stores
108,Wyoming,2021,20.0,Starbucks Stores
109,North Dakota,2023,8.0,Starbucks Stores


### Appendix and References

https://worldpopulationreview.com/state-rankings/starbucks-stores-by-state

https://worldpopulationreview.com/state-rankings/dunkin-donuts-by-state

https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population

https://worldpopulationreview.com/state-rankings/chick-fil-a-by-state

https://en.wikipedia.org/wiki/List_of_regions_of_the_United_States 

https://finance.yahoo.com/quote/SBUX/

https://www.investing.com/equities/dunkin-brands-group


Generative A.I. Statement: Chat-GPT was used to suggest changes in code to debug errors. An example of errors include, incorrect syntax, incorrect usage of parameters for plots, making suggestions to fix parameters to resolve the error in code. Chat-GPT was only used to resolve errors in already hand written code. 